# Options_Trading_6

For this assignment you can use the functions in the black_scholes.py module, which should be included with this assignment.

In [3]:
import black_scholes as bs

### Dummy data

In [4]:
from client.info_messages import PriceBook, PriceVolume

#stocks pricebook 
pb = PriceBook()
pb.instrument_id = 'AAPL'
pb.bids = [PriceVolume(99, 100)]
pb.asks = [PriceVolume(101, 100)]

#my added input 
# Option pricebook
pb_a = PriceBook()
pb_a.instrument_id = 'AAPL_CALL'
pb_a.bids = [PriceVolume(6, 100)]
pb_a.asks = [PriceVolume(10, 100)]

pb_b = PriceBook()
pb_b.instrument_id = 'AAPL_PUT'
pb_b.bids = [PriceVolume(8, 100)]
pb_b.asks = [PriceVolume(11, 100)]

pb_c = PriceBook()
pb_c.instrument_id = 'AAPL_PUT'
pb_c.bids = [PriceVolume(8, 100)]
pb_c.asks = [PriceVolume(11, 100)]



# Dummy data
call = Option('call', 1, 100)
put = Option('put', 0.5, 110)

call.kind  # call or put
call.tte  # time to expiry
call.strike  # strike price

option_positions = [(call, 100), (put, -50)]

S = 100  # Underlying price
r = 0  # Interest rate
sigma = 0.2  # Volatility

from scipy import stats
import numpy as np

_norm_cdf = stats.norm(0, 1).cdf
_norm_pdf = stats.norm(0, 1).pdf


def _d1(S, K, T, r, sigma):
    return (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))


def _d2(S, K, T, r, sigma):
    return _d1(S, K, T, r, sigma) - sigma * np.sqrt(T)

def call_value(S, K, T, r, sigma):
    return S * _norm_cdf(_d1(S, K, T, r, sigma)) - K * np.exp(-r * T) * _norm_cdf(_d2(S, K, T, r, sigma))

def put_value(S, K, T, r, sigma):
    return np.exp(-r * T) * K * _norm_cdf(-_d2(S, K, T, r, sigma)) - S * _norm_cdf(-_d1(S, K, T, r, sigma))

def call_delta(S, K, T, r, sigma):
    return _norm_cdf(_d1(S, K, T, r, sigma))

def put_delta(S, K, T, r, sigma):
    return call_delta(S, K, T, r, sigma) - 1


def call_vega(S, K, T, r, sigma):
    return S * _norm_pdf(_d1(S, K, T, r, sigma)) * np.sqrt(T)

def put_vega(S, K, T, r, sigma):
    return call_vega(S, K, T, r, sigma)

def best_bid_price(PriceBook):
    
    if PriceBook.bids == []:
        return 
    return sorted(PriceBook.bids, key = lambda x: x.price)[-1].price

def best_ask_price(PriceBook):
    
    if PriceBook.asks == []:
        return 
    return sorted(PriceBook.asks, key = lambda x: x.price)[0].price

### Option Vega

Write a function that calculates the black-scholes option vega given an option and market data.

input: Option, S, r, sigma

output: option vega

In [73]:
def option_vega(option, S, r, sigma):
    
    if option.kind == 'call':
        K = option.strike
        T = option.tte
        c_vega = call_vega(S, K, T, r, sigma)
        return c_vega
    
    if option.kind == 'put':
        K = option.strike
        T = option.tte
        p_vega = put_vega(S, K, T, r, sigma)
        return p_vega
    
option_vega(call, S, r, sigma)

39.69525474770118

### Cheapest Vega

Your function receives three options with their pricebook as input.
The goal of the function is to determine which option would be the cheapest vega hedge.

Assume you currently have a vega position (size and direction is irrelevant in this exercise). Your risk departement wants you to reduce this position.
Your function should determine which option is the cheapest to use in reducing your risk.

For the purposes of this exercise we value the options at the midprice of each of their books.
This means that the cost of buying/selling an option is half the bid/ask spread.

Note that different options have different vegas, so the cheapest option is not neccesarily the one with the smallest bid/ask spread.

For this exercise you can assume all you books will have at least one bid and one ask.

For example: Imagine a book of an option is 8.5 @ 11.5 (best_bid@best_ask) and the vega of that option is 3. The cost of buying 1 lot of this option is 1.5 as we buy it for 11.5 while it is worth 10 (midprice). As the vega is 3, per vega we pay 0.5. In other words, per euro we get 2 vegas.

Doing this analyis for each option book should allow you to determine the cheapest one.

input: Option, PriceBook, Option, PriceBook, Option, PriceBook, S, r, sigma

output: instrument_id of the cheapest instrument (a for option_a, b for option_b and c for option_c) (these can also be found from the PriceBook: pb.instrument_id)

In [72]:
#vega: point change in theoretical value of an option (bc) for each one percentage point change in volatility 

def cheapest_vega(option_a, pb_a, option_b, pb_b, option_c, pb_c, S, r, sigma):
    
    best_bid_a = best_bid_price(pb_a)
    best_ask_a = best_ask_price(pb_a)
    cost_a = (best_ask_a - best_bid_a) / 2 
    a = 1/cost_a
    
    best_bid_b = best_bid_price(pb_b)
    best_ask_b = best_ask_price(pb_b)
    cost_b = (best_ask_b - best_bid_b) / 2 
    b = 1/cost_b
    
    best_bid_c = best_bid_price(pb_c)
    best_ask_c = best_ask_price(pb_c)
    cost_c = (best_ask_c - best_bid_c) / 2
    c = 1/cost_c
    
    max_ = a
    if b > max_:
        max_ = b
        return pb_b.instrument_id
    if c > max_:
        max_ = c 
        return pb_c.instrument_id
    else:
        return pb_a.instrument_id
    
cheapest_vega(put, pb_a, call, pb_b, put, pb_c, S, r, sigma)    

'AAPL_PUT'

### Finding implied volatility

Write a function that calculates the implied volatility of the option using Newton's method.

https://en.wikipedia.org/wiki/Newton%27s_method

The implied volatility of a given option price is the volatility you need to put in the black-scholes formula so that the given option price comes out.
There is no analytical formula for the inverse so we do it using a numerical root finding method (here Newton's method).

Stop the optimisation if you've reached 100 iterations or if the abs difference between successive updates is smaller than 1e-05 (0.00001).

input: Option, price, S, r

output: implied volatility

In [77]:
def option_price(option, S, r, sigma):
    
    if option.kind == 'call':
        K = option.strike
        T = option.tte
        f = bs.call_value(S, K, T, r, sigma)
        return f
    
    if option.kind == 'put':
        K = option.strike
        T = option.tte
        f = bs.put_value(S, K, T, r, sigma)
        return f
    
#print(option_price(call, S, r, sigma))

def f(option, S, r, sigma, price):
    
        return option_price(option, S, r, sigma) - price 
    
#print(f(call, S, r, sigma, 0.5))
    
def derivative(function, option, S, r, sigma, price):
    return (function(option, S, r, sigma + 1e-05, price) - function(option, S, r, sigma - 1e-05, price)) /2e-05 

f(call, S, r, 0, 5)/derivative(f, call, S, r, 0, 5)
    
def implied_vol(option, price, S, r):

    old_sigma = 0.2
    new_sigma = 0.2
    for i in range(100):
        new_sigma = old_sigma - f(option, S, r, old_sigma, price)/ derivative(f, option, S, r, old_sigma, price)
        if abs(old_sigma-new_sigma) <= 1e-05:
            break 
        old_sigma = new_sigma
    return new_sigma

implied_vol(call, 0.5, S, r)


0.01253322340350088